In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, Callback, LearningRateScheduler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, CuDNNLSTM, Embedding, Dropout, TimeDistributed, CuDNNLSTM, Activation, Input, GRU, Flatten, Softmax, RepeatVector, Permute, Multiply, Bidirectional

import numpy as np
import random
import sys, re, io, math
import string, os 
from google.colab import drive
from google.colab import files

In [0]:
files.upload()

#Word model

In [0]:
text = []

specials = {'\u200b': ' ', 
              '…': ' ... ', 
              '\ufeff': '',
              '\xa0': '',
              '❒':'',
              '\n': ' ',
              '–': ' ',
              '«': '',
              '»': '',                        
           }


#read file
#with open(path, encoding='Cp1251') as fp:  
with open(path) as fp:  
  line = fp.readline()
  while line:   
    #replace special symbols
    for ch in specials:
      line = line.replace(ch,specials[ch])
    #clean punctuation
    line = "".join(v for v in line if v not in string.punctuation).lower()
    #clean duplicate space
    line = re.sub(' +', ' ',line)
    text.append(line)
    line = fp.readline()
    

In [112]:
len(text)

4279

###text 100

In [26]:
text[:100]

[' просыпается студент утром в голове шум и одна мысль ',
 ' чёрт сегодня же экзамен ',
 ' коекак добирается до института находит нужную аудиторию никого не узнавая берёт билет садится за стол понимает что не может ответить ни на один вопрос спрашивает слева справа списывает спереди и сзади в конце экзамена подходит к преподавателю и отдаёт ему свои бумажки ',
 ' ладно за то что вы здесь написали поставлю вам три балла устроит ',
 ' вполне ',
 ' давайте зачётку многое я в жизни повидал но чтобы второкурсник сдавал экзамен за третий курс никогда ',
 ' встречаются четыре студента один говорит ',
 ' мужики давайте сегодня вечером ко мне есть пиво рыба ',
 ' а девчонки будут ',
 ' и с девчонками я уже договорился их сегодня не будет ',
 ' давайте просто посидим молча здесь так красиво не хочу нарушать эту тишину ',
 ' тяните билет ',
 ' радостный студент выскакивает изза стола и бежит к выходу ',
 ' сдал ',
 ' постойте кричит профессор указывая на зачётку надо же отметить ',
 ' вечером отм

###tokenize

In [29]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(text)
inp_sequences[:10]


[[759, 46],
 [759, 46, 318],
 [759, 46, 318, 2],
 [759, 46, 318, 2, 760],
 [759, 46, 318, 2, 760, 761],
 [759, 46, 318, 2, 760, 761, 1],
 [759, 46, 318, 2, 760, 761, 1, 155],
 [759, 46, 318, 2, 760, 761, 1, 155, 637],
 [1814, 106],
 [1814, 106, 26]]

In [113]:
total_words

9539

In [0]:
#vectorize sequences
import keras.utils as ku 

def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

##Model

###LSTM model

In [93]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 89, 100)           953900    
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 768)               2672640   
_________________________________________________________________
dropout_21 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 9539)              7335491   
Total params: 10,962,031
Trainable params: 10,962,031
Non-trainable params: 0
_________________________________________________________________


In [92]:
input_len = max_sequence_len - 1
model = Sequential()

model.add(Embedding(total_words, 100, input_length=input_len))
model.add(CuDNNLSTM(768))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

checkpoint = ModelCheckpoint('nlp.hfd5', monitor='loss',verbose=0, save_best_only=True,mode='min')

history = model.fit(predictors, label, epochs=100, verbose=1, callbacks=[checkpoint])

Epoch 1/100
28198/28198 [==============================] - 24s 855us/sample - loss: 8.0638
Epoch 2/100
28198/28198 [==============================] - 22s 780us/sample - loss: 7.5584
Epoch 3/100
28198/28198 [==============================] - 22s 789us/sample - loss: 7.1620
Epoch 4/100
28198/28198 [==============================] - 23s 799us/sample - loss: 6.5858
Epoch 5/100
28198/28198 [==============================] - 23s 802us/sample - loss: 5.7155
Epoch 6/100
28198/28198 [==============================] - 23s 808us/sample - loss: 4.5963
Epoch 7/100
28198/28198 [==============================] - 23s 812us/sample - loss: 3.4360
Epoch 8/100
28198/28198 [==============================] - 23s 817us/sample - loss: 2.5575
Epoch 9/100
28198/28198 [==============================] - 23s 821us/sample - loss: 1.9784
Epoch 10/100
28198/28198 [==============================] - 23s 819us/sample - loss: 1.5888
Epoch 11/100
28198/28198 [==============================] - 23s 824us/sample - loss: 1.32

###Bidirectional model

In [107]:
input_len = max_sequence_len - 1
model = Sequential()

model.add(Embedding(total_words, 100, input_length=input_len))
model.add(Bidirectional(CuDNNLSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


#model.load_weights('nlpbd.hfd5')
checkpoint = ModelCheckpoint('nlpbd.hfd5', monitor='loss',verbose=0, save_best_only=True,mode='min')

history = model.fit(predictors, label, epochs=100, verbose=1, callbacks=[checkpoint])

Epoch 1/100
28198/28198 [==============================] - 15s 518us/sample - loss: 0.4964
Epoch 2/100
28198/28198 [==============================] - 12s 429us/sample - loss: 0.4873
Epoch 3/100
28198/28198 [==============================] - 12s 439us/sample - loss: 0.4820
Epoch 4/100
28198/28198 [==============================] - 12s 432us/sample - loss: 0.4775
Epoch 5/100
28198/28198 [==============================] - 12s 425us/sample - loss: 0.4791
Epoch 6/100
28198/28198 [==============================] - 12s 430us/sample - loss: 0.4758
Epoch 7/100
28198/28198 [==============================] - 12s 431us/sample - loss: 0.4690
Epoch 8/100
28198/28198 [==============================] - 12s 428us/sample - loss: 0.4680
Epoch 9/100
28198/28198 [==============================] - 12s 425us/sample - loss: 0.4701
Epoch 10/100
28198/28198 [==============================] - 12s 429us/sample - loss: 0.4656
Epoch 11/100
28198/28198 [==============================] - 12s 427us/sample - loss: 0.46

##Predict

###LSTM

In [0]:
model.load_weights('nlp.hfd5')

In [99]:

def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text#.title()

  
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def temperature_generate_text(temperature, seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        prediction = model.predict(token_list, verbose=0)[0]
        predicted = sample(prediction, temperature)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text#.title()
print (generate_text("стоит штирлиц на крымском мосту", 300, model, max_sequence_len))  
print (temperature_generate_text(.7, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))    
print (temperature_generate_text(.9, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))    
print (temperature_generate_text(1.2, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))   
print (temperature_generate_text(1.7, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))   

стоит штирлиц на крымском мосту посту солдат проходит мимо майор и говорит жене вы должны чистосердечно рассказать и так и даже не знаю что вам или значит его только но и опасно всех глаз в ад камеру они он меня по быстрому но аккуратненько без жертв ты же десантура умеешь бутылку домой и не досчитались и турнули в конце концов я когда и тоже научусь ехать на мотоцикле и буду показывать такой аттракцион мудрая мама отвечает боренька еврей на мотоцикле это уже аттракцион зачем еще по стенкам ехать и сзади подходит к нему подходит к девушкой и отдаёт ему свои бумажки а еще и могу блондинка на мотоцикле это наш аттракцион зачем еще по стенкам ехать и уходит и стоит оборотней а вы его на смех поднял мол как это тебя жена сделала и так далее и тому подобное мол я со своей жмет на мотоцикле и буду показывать бокал и сообщает под бокал через пять минут наш снова разворачивает жмет и кричит отвечает на мотоцикле и один на себя история повторяется и снова служанка остается а еще через несколько

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


стоит штирлиц на крымском мосту посту солдат проходит мимо майор и говорит себе вы говорит он байкеру он же профессор и так далее сара тому бог жмет на стартер мотор взвывает и атеизма катит бокал кто отец это напоследок ему принесли вина а еще из мотоциклистов да там в конце экзамена я и тоже тоже ехать на мотоцикле и какойто в ад одна с женой врач с их маслом вокруг себя спят рассказать в мире дело с моим чешет если все здесь ее напоследок в месяц и убить из пистолета собственную жену но свинья это тут с их ехать на мотоцикле и будете в кювет второй джип резко тормозит сбивает старушку и переворачивается в него врезается лимузин мотоциклисты падают и рассыпаются по залитой горящим бензином и усыпанной стеклами дороге автобус со спецохраной переезжает мотоциклистов и завалившись на бок взрывается шум дым крики битый металл стекло один берет паспорт и говорит не знаю всё это беременна и опять не знаю бы это или нет состояния о дома и на это дело с профессорской полной квартире и сообща

###Bidirectional

In [0]:
model.load_weights('nlpbd.hfd5')

In [110]:
print (generate_text("стоит штирлиц на крымском мосту", 300, model, max_sequence_len))  
print (temperature_generate_text(.7, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))    
print (temperature_generate_text(.9, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))    
print (temperature_generate_text(1.2, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))   
print (temperature_generate_text(1.7, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))   
print (temperature_generate_text(10, "стоит штирлиц на крымском мосту", 300, model, max_sequence_len))   

стоит штирлиц на крымском мосту посту солдат проходит мимо майор и говорит другому как фильм умный этих давайте пожалуйста ну я как было в это время еврей ещё и было мол и это тебя эти мотоциклисты берутся дайте мне двойное виски напоследок ему принесли виски он выпил и с криком да здравствуют сша выбросился выходит русский дайте мне водки напоследок выпил еще бутылку выпил еще бутылку он допил третью бутылку и с криком да здравствуют сша выбросился выходит русский дайте мне водки напоследок выпил еще бутылку выпил еще бутылку он допил третью бутылку и и говорит говорит и головой новый русский говорит и говорит и говорит приползает и вокруг говорит афроамериканцев и паря и паря говорит купе купе паря и вкусных русский говорит говорит москву эту афроамериканцев слух москву животных федя и да и говорит машина охотничьи поезда сбегать москву кажется или говорит италии таня я паря оглядывается и сколько говорит вокруг вокруг закричишь слух мне сына пока и паря москву или паря говорит москв

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


стоит штирлиц на крымском мосту посту солдат проходит мимо майор и говорит другому как ставит на это та не делается он не помогает найти сходить в это время ему сходить в кино а это как как ты же голова больше моя девушка дайте мне двух это второй жену в одной и попытками в кабинет по залитой горящим бензином и усыпанной стеклами дороге автобус со спецохраной переезжает мотоциклистов и завалившись на бок взрывается шум дым крики битый металл стекло из одной машины выползает новый русский трясет головой оглядывается вокруг и головой к головой пожалуйста лежит и говорит говорит русский говорит головой и говорит да и говорит диалог и да и говорит москву говорит выводят того купил и италии да пока жена говорит афроамериканцев и говорит тебя и паря зарплату и говорит да и паря хохол и да и паря говорит выводят медведя да и говорит говорит выводят говорит афроамериканцев москву час и говорит ереван охотничьи африка хочет да иван и превысили паря минут приползает ваши говорит и вокруг говорит

In [111]:
print (temperature_generate_text(.7, "стоит штирлиц на крымском", 300, model, max_sequence_len))    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


стоит штирлиц на крымском посту солдат проходит мимо майор и говорит говорит другому как шо он фильм понравился как чем ну вот он говорит довозят до себя да его два ещё утром в газовую камеру и дал мешок сухарей и овчарку кто больше и мотоциклистов и денег дают и русский и мне виски но он отвечает выпил и мной с одной это ассоциируете с кем ты русский уже мотоцикле водки по одной металл стекло и по собаками из одной миски жрал но не выдержал и головой к афроамериканцев и вокруг говорит плохо говорит парень говорит присылали и говорит да и говорит присылали говорит да москву машины да и говорит подтянув хохол русский час и говорит учёные нем афроамериканцев слух да слух говорит москву эту пятницу слух и слух поднимает говорит афроамериканцев и слух говорит пьяный говорит и превысили говорит русский и говорит вас москву уменя да слух и зарплату вчера можете выкинул для афроамериканцев слух говорит охотничьи поезде в паря кажется и говорит без охотничьи подруги для говорит подруги говорит

In [131]:
for i in range(10):
  t = np.random.rand()  
  print (t, temperature_generate_text(t, text[np.random.randint(1,len(text),1)[0]], 100, model, max_sequence_len))    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


0.21176360259634264  да я его туда ппперенес  50 просто водку в стакан и прорубь чтобы не тут такой пользоваться компьютером он тем что у меня было голова болит и кричит отвечает своему моя их к нему подходит к нему выходит спит и в мире узнавать парень и уволены через несколько месяцев служанка говорит хозяйке на мотоцикле и говорит еще на мотоцикле и пьёт на мотоцикле и буду показывать такой аттракцион мудрая мама отвечает боренька еврей на мотоцикле и уже аттракцион зачем еще русский трясет головой к афроамериканцев и вокруг и вокруг говорит подруги говорит говорит после говорит хохол и выводят говорит выводят говорит и ветврач говорит москву
0.8736504028551557  лев собрал всех зверей и говорит  себе генерал лошадь падает спрашивает у ему 3 машину 000 в китай сказано сделано и спрашивает к двум котлам с кипящим маслом вокруг одного все черти столпились а вокруг другого никого два окна моя экзамена под одеяло и уходит на мотоцикле мама еврей про это такой еврей на мотоцикле и буду по

#Домашнее задание

В этом домашнем задании вам нужно сгенерировать интересный текст с помощью LSTM.

Выберите интересный источник для обучения (https://www.gutenberg.org/, http://lib.ru/ ).
Подберите архитектуру LSTM и количество эпох обучения.
Попробуйте использовать разную "температуру" при генерации текстов.
Лучший текст, который вам удастся получить, пришлите кураторам со ссылкой на ноутбук с кодом решения генерации этого текста.

#Вывод

[colab](https://colab.research.google.com/drive/1D2LZzS-Ag0a6cltmdRRVla42AP8sQlIj)

В качестве обучающей выборки  использовал сборник анекдотов из 4279 строк, 9539 слов, для обучения использовал LSTM и Bidirectional сеть. В качестве примера предсказания использовалась сеть LSTM c температурой 0.9

стоит штирлиц на крымском мосту посту солдат проходит мимо майор и говорит себе вы говорит он байкеру он же профессор и так далее сара тому бог жмет на стартер мотор взвывает и атеизма катит бокал кто отец это напоследок ему принесли вина а еще из мотоциклистов да там в конце экзамена я и тоже тоже ехать на мотоцикле и какойто в ад одна с женой врач с их маслом вокруг себя спят рассказать в мире дело с моим чешет если все здесь ее напоследок в месяц и убить из пистолета собственную жену но свинья это тут с их ехать на мотоцикле и будете в кювет второй джип резко тормозит сбивает старушку и переворачивается в него врезается лимузин мотоциклисты падают и рассыпаются по залитой горящим бензином и усыпанной стеклами дороге автобус со спецохраной переезжает мотоциклистов и завалившись на бок взрывается шум дым крики битый металл стекло один берет паспорт и говорит не знаю всё это беременна и опять не знаю бы это или нет состояния о дома и на это дело с профессорской полной квартире и сообщает ему на собой ехать наш берет один и кричит отвечает бокал и со мировой конём голос месяцев отвечает ее и поинтересовался в ад дело у студентов ночь одна муж но в мире узнавать вокруг и снова служанка сбивает старушку но переворачивается в него врезается лимузин мотоциклисты падают и рассыпаются по залитой горящим бензином и усыпанной стеклами дороге автобус со спецохраной переезжает мотоциклистов и завалившись на бок взрывается шум дым крики битый металл стекло за одной гвоздь выползает новый тут и спрашивает были него вдруг утверждение но еще почему тут снова двух первый на выходит выходит старушка с облегчением мол победили и тут один из них интересуется но еще тебя стоит блондинка на месте я я говорит но там его вместо ругани но еще и с криком да здравствует франция
